# Imports

In [3]:
# System Imports
import os
import time

# Data Imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy as sp

import helper_functions as hf

# Deep Learning Framework
import tensorflow as tf

# SSL
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
for device in tf.config.list_physical_devices('GPU'):
    print(f"* {device}")

## Reading in Images

In [5]:
# Setting paths

train_data_path = './data/archive/train'
test_data_path = './data/archive/test'

In [6]:
# Viewing the classes

categories = os.listdir(train_data_path)
print(f"The classes are: {categories}")

The classes are: ['daisy', 'rose', 'tulip', 'dandelion', 'sunflower']


In [7]:
# Configurations

SEED = 0
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.20
EPOCHS = 10

In [8]:
# Creating the training set

training_set = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    class_names=categories
)

Found 2746 files belonging to 5 classes.
Using 2197 files for training.


In [9]:
# Creating the validation set

validation_set = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    class_names=categories
)

Found 2746 files belonging to 5 classes.
Using 549 files for validation.


## Plotting Images

In [10]:
# Plotting some images

# hf.plot_images(training_set, categories)

# Modeling

In [11]:
# Function that creates a model

def get_baseline_model(): 

    ## Clearing backend
    tf.keras.backend.clear_session()

    ## Input Layer
    inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

    # Rescaling the images
    x = tf.keras.layers.Rescaling(1./255)(inputs)

    ## First CNN
    x = tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(3,3),
        strides=(1,1),
        activation='relu',
    )(x)

    x = tf.keras.layers.MaxPool2D(
        pool_size=(2, 2)
    )(x)

    ## Second CNN
    x = tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3,3),
        strides=(1,1),
        activation='relu',
    )(x)

    x = tf.keras.layers.MaxPool2D(
        pool_size=(2, 2)
    )(x)

    ## Third CNN
    x = tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3,3),
        strides=(1,1),
        activation='relu',
    )(x)

    x = tf.keras.layers.MaxPool2D(
        pool_size=(2, 2)
    )(x)

    ## Flatten layer
    x = tf.keras.layers.Flatten()(x)

    ## First Dense layer
    x = tf.keras.layers.Dense(
        units=64,
        activation='relu'
    )(x)

    ## Output
    outputs = tf.keras.layers.Dense(
        units=len(categories),
        activation='softmax'
    )(x)

    ## Creating Model
    model = tf.keras.Model(
        inputs=inputs,
        outputs=outputs
    )

    ## Compiling the model
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )

    ## Viewing the architecture
    model.summary()

    return model

In [12]:
model_elapsed_time = {}

In [13]:
# # Training the model

# start_time = time.time()

# baseline_model = get_baseline_model()

# history = baseline_model.fit(
#   training_set,
#   validation_data=validation_set,
#   epochs=EPOCHS
# )

# elasped_time = time.time() - start_time
# model_elapsed_time["baseline"] = elasped_time

In [14]:
# # Viewing the results of the training

# hf.plot_history(history)

In [15]:
# hf.plot_actual_prediction(baseline_model, categories, validation_set)

Model is not performing well. Accuracy is terrible (bias is high). And generalization is bad (variance is high). 

Since the model did not accurately predict the flowers, the model didn't learn key features that differentiates the flowers (categories).

Solutions:
<ul>
    <li> Data Augmentation </li>
         * Create new images by augmenting the images to expose the model to more images
    <li> Transfer Learning</li>
        * Use a working model that performs well for our task<br>
    <li> Get more data</li>
        * Find more data for the model
</ul>

## Data Augmentation

In [16]:
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip(
      "horizontal_and_vertical", 
      input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3)
  ),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
  tf.keras.layers.RandomCrop(
    128, 128
  ),
  tf.keras.layers.Resizing(IMAGE_SIZE[0], IMAGE_SIZE[0])
])

def prepare(ds, shuffle=False, augment=False):

    if shuffle:
        ds = ds.shuffle(1000)

    # Use data augmentation only on the training set.
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y), 
            num_parallel_calls=AUTOTUNE
        )

    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=AUTOTUNE)

In [17]:
train_ds = prepare(training_set, shuffle=True, augment=True)
val_ds = prepare(validation_set)

In [18]:
# # Running the model

# tf.keras.backend.clear_session()

# start_time = time.time()

# augment_model = get_baseline_model()

# augment_history = augment_model.fit(
#     train_ds,
#     validation_data = val_ds,
#     epochs=EPOCHS
# )

# elasped_time = time.time() - start_time
# model_elapsed_time['data_augmentation'] = elasped_time

In [19]:
# # Viewing the results of the training

# hf.plot_history(augment_history)

In [20]:
# augment_model.save('saved_model/augment_model.h5')

The training and validation have similar accuracy and loss (low variance). However the accuracy is ~70% (high bias). If we would like, we can do more augmentation like random contrasting.

Next we will try transfer learning. Use a model that has a great track record classifying and apply it to our task.

## Transfer Learning

In [21]:
# base_transfer_model = tf.keras.applications.MobileNetV3Small(
#     input_shape=IMAGE_SIZE+ (3,),
#     include_top=False,
#     weights='imagenet',
# )

# base_transfer_model.trainable = False

# base_transfer_model.summary()

In [22]:
# image_batch, label_batch = next(iter(train_ds))
# feature_batch = base_transfer_model(image_batch)
# print(feature_batch.shape)

In [23]:
# tf.keras.backend.clear_session()
# inputs = tf.keras.Input(shape=(224, 224, 3))
# x = data_augmentation(inputs)
# x = base_transfer_model(x, training=False)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# outputs = tf.keras.layers.Dense(len(categories), activation='softmax')(x)
# transfer_model = tf.keras.Model(inputs, outputs)

In [24]:
# # start_time = time.time()

# transfer_model.compile(
#     optimizer='adam',
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=['accuracy']
# )

# # elasped_time = time.time() - start_time
# # model_elapsed_time['transfer_model'] = elasped_time

In [25]:
# transfer_model.summary()

In [26]:
# transfer_history = transfer_model.fit(
#     train_ds,
#     validation_data = val_ds,
#     epochs=EPOCHS
# )

In [27]:
# # Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(base_transfer_model.layers))

# # Fine-tune from this layer onwards
# fine_tune_at = 200

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_transfer_model.layers[fine_tune_at:]:
#     layer.trainable = True

In [28]:
# transfer_model.compile(
#     optimizer='adam',
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=['accuracy']
# )

# transfer_fine = transfer_model.fit(
#     train_ds,
#     epochs=EPOCHS+10,
#     initial_epoch=transfer_history.epoch[-1],
#     validation_data=val_ds
# )

In [29]:
# hf.plot_history(transfer_fine)

In [30]:
# transfer_model.save('saved_model/transfer_model.h5')

In [31]:
# from tenkeras.models import load_model
# loaded_model = tf.keras.models.load_model("./saved_model/transfer_model.h5")

In [32]:
# from PIL import Image

# new_image_path = './data/FREEDOM.jpg'
# image = Image.open(new_image_path).resize((224,224))

In [33]:
# arr = np.expand_dims(tf.keras.preprocessing.image.img_to_array(image), axis=0)

In [34]:
# pred = loaded_model.predict(arr)

In [35]:
# np.argmax(pred, axis=1)

In [36]:
# resnet_model = tf.keras.applications.resnet50.ResNet50(
#     input_shape=IMAGE_SIZE+ (3,),
#     include_top=False,
#     weights='imagenet',
# )

# resnet_model.trainable = False

# # Let's take a look to see how many layers are in the base model
# print("Number of layers in the base model: ", len(resnet_model.layers))

In [37]:
# resnet_model.trainable = False

# # Fine-tune from this layer onwards
# fine_tune_at = 170

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in resnet_model.layers[fine_tune_at:]:
#     layer.trainable = True

In [38]:
# tf.keras.backend.clear_session()

# mirrored_strategy = tf.distribute.MirroredStrategy()

In [39]:
# # Create Distributed Datasets from the datasets
# train_ds = mirrored_strategy.experimental_distribute_dataset(train_ds)
# val_ds = mirrored_strategy.experimental_distribute_dataset(val_ds)

In [40]:
# def create_model():

#     inputs = tf.keras.Input(shape=(224, 224, 3))
#     x = resnet_model(inputs, training=False)
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Flatten()(x)
#     x = tf.keras.layers.Dense(256)(x)
#     x = tf.keras.layers.Dropout(0.5)(x)
#     outputs = tf.keras.layers.Dense(len(categories), activation='softmax')(x)
    
#     return tf.keras.Model(inputs, outputs)

In [41]:
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='val_loss', 
#     factor=0.2,
#     patience=2, 
#     min_lr=0.001
# )

# # es = tf.keras.callbacks.EarlyStopping(
# #     monitor='val_loss',
# #     min_delta=0,
# #     patience=3,
# #     verbose=0,
# #     mode='auto',
# #     baseline=None,
# #     restore_best_weights=False,
# # )

# with mirrored_strategy.scope():
    
#     resnet_model = tf.keras.applications.resnet50.ResNet50(
#         input_shape=IMAGE_SIZE+ (3,),
#         include_top=False,
#         weights='imagenet',
#     )

#     resnet_model.trainable = False
    
#     inputs = tf.keras.Input(shape=(224, 224, 3))
#     x = resnet_model(inputs, training=False)
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
#     x = tf.keras.layers.Flatten()(x)
#     x = tf.keras.layers.Dense(256)(x)
#     x = tf.keras.layers.Dropout(0.5)(x)
#     outputs = tf.keras.layers.Dense(len(categories), activation='softmax')(x)
    
#     transfer_resnet_model = tf.keras.Model(inputs, outputs)
    
#     transfer_resnet_model.compile(
#         optimizer='adam',
#         loss=tf.keras.losses.CategoricalCrossentropy(),
#         metrics=['accuracy']
#     )

#     transfer_fine = transfer_resnet_model.fit(
#         train_ds,
#         epochs=20,
#         # initial_epoch=transfer_history.epoch[-1],
#         validation_data=val_ds,
#         callbacks=[reduce_lr]
#     )

In [42]:
tf.keras.backend.clear_session()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=2, 
    min_lr=0.001
)

resnet_model = tf.keras.applications.resnet50.ResNet50(
    input_shape=IMAGE_SIZE+ (3,),
    include_top=False,
    weights='imagenet',
)

resnet_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = resnet_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(len(categories), activation='softmax')(x)

transfer_resnet_model = tf.keras.Model(inputs, outputs)

transfer_resnet_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

transfer_fine = transfer_resnet_model.fit(
    train_ds,
    epochs=20,
    # initial_epoch=transfer_history.epoch[-1],
    validation_data=val_ds,
    callbacks=[reduce_lr]
)

2022-12-20 13:31:43.661884: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20
69/69 [==============================] - 90s 1s/step - loss: 1.6587 - accuracy: 0.6468 - val_loss: 0.7771 - val_accuracy: 0.8215 - lr: 0.0010
Epoch 2/20
69/69 [==============================] - 86s 1s/step - loss: 0.9476 - accuracy: 0.7487 - val_loss: 0.5969 - val_accuracy: 0.8397 - lr: 0.0010
Epoch 3/20
69/69 [==============================] - 87s 1s/step - loss: 0.7773 - accuracy: 0.7802 - val_loss: 0.7426 - val_accuracy: 0.8160 - lr: 0.0010
Epoch 4/20
69/69 [==============================] - 88s 1s/step - loss: 0.7685 - accuracy: 0.7883 - val_loss: 0.7527 - val_accuracy: 0.8251 - lr: 0.0010
Epoch 5/20
69/69 [==============================] - 86s 1s/step - loss: 0.6641 - accuracy: 0.8061 - val_loss: 0.5508 - val_accuracy: 0.8470 - lr: 0.0010
Epoch 6/20
69/69 [==============================] - 85s 1s/step - loss: 0.6898 - accuracy: 0.8093 - val_loss: 0.6795 - val_accuracy: 0.8361 - lr: 0.0010
Epoch 7/20
69/69 [==============================] - 86s 1s/step - loss: 0.6074 - a

KeyboardInterrupt: 